In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from sympy.physics.quantum import TensorProduct

In [ ]:
def indf(i,j):
    return i*Ny*Nbands + j*Nbands

In [ ]:
def flake_geometry(h_symbolic, syms, Nx, Ny, Nbands):
    kx_sym, ky_sym = syms 
    
    Lx_nn_pos = sp.integrate(h_symbolic * sp.exp(sp.I * kx_sym * (-1.0)), (kx_sym, -sp.pi, sp.pi))/ (2 * sp.pi)
    Lx_nn_neg = sp.integrate(h_symbolic * sp.exp(sp.I * kx_sym), (kx_sym, -sp.pi, sp.pi))/ (2 * sp.pi)
    Lx_nnn_pos = sp.integrate(h_symbolic * sp.exp(sp.I * 2 * kx_sym * (-1.0)), (kx_sym, -sp.pi, sp.pi))/ (2 * sp.pi)
    Lx_nnn_neg = sp.integrate(h_symbolic * sp.exp(sp.I * 2 * kx_sym), (kx_sym, -sp.pi, sp.pi))/ (2 * sp.pi)

    Ly_nn_pos = sp.integrate(h_symbolic * sp.exp(sp.I * ky_sym * (-1.0)), (ky_sym, -sp.pi, sp.pi))/ (2 * sp.pi)
    Ly_nn_neg = sp.integrate(h_symbolic * sp.exp(sp.I * ky_sym), (ky_sym, -sp.pi, sp.pi))/ (2 * sp.pi)
    Ly_nnn_pos = sp.integrate(h_symbolic * sp.exp(sp.I * 2 * ky_sym * (-1.0)), (ky_sym, -sp.pi, sp.pi))/ (2 * sp.pi)
    Ly_nnn_neg = sp.integrate(h_symbolic * sp.exp(sp.I * 2 * ky_sym), (ky_sym, -sp.pi, sp.pi))/ (2 * sp.pi)

    Lx_nn_pos = Lx_nn_pos.rewrite(sp.cos).simplify()
    Lx_nn_neg = Lx_nn_neg.rewrite(sp.cos).simplify()
    Lx_nnn_pos = Lx_nnn_pos.rewrite(sp.cos).simplify()
    Lx_nnn_neg = Lx_nnn_neg.rewrite(sp.cos).simplify()
    Ly_nn_pos = Ly_nn_pos.rewrite(sp.cos).simplify()
    Ly_nn_neg = Ly_nn_neg.rewrite(sp.cos).simplify()
    Ly_nnn_pos = Ly_nnn_pos.rewrite(sp.cos).simplify()
    Ly_nnn_neg = Ly_nnn_neg.rewrite(sp.cos).simplify()

    H_diag = h_symbolic - (Lx_nn_pos * sp.exp(sp.I * kx_sym) + Lx_nn_neg * sp.exp(-sp.I * kx_sym) + Lx_nnn_pos * sp.exp(sp.I * 2 * kx_sym) + Lx_nnn_neg * sp.exp(-sp.I * 2 * kx_sym))
    H_diag -= (Ly_nn_pos * sp.exp(sp.I * ky_sym) + Ly_nn_neg * sp.exp(-sp.I * ky_sym) + Ly_nnn_pos * sp.exp(sp.I * 2 * ky_sym) + Ly_nnn_neg * sp.exp(-sp.I * 2 * ky_sym))
    H_diag = H_diag.rewrite(sp.cos).simplify()
    H_diag = sp.nsimplify(H_diag, tolerance = 1e-8)

    h = sp.zeros(Nx*Ny*Nbands, Nx*Ny*Nbands)

    for i in range(Nx):
        for j in range(Ny):
            h[indf(i,j):indf(i,j+1),indf(i,j):indf(i,j+1)] = H_diag
            
            if i > 0: 
                h[indf(i-1,j):indf(i-1,j+1), indf(i,j):indf(i,j+1)] = Lx_nn_pos
                
            if i > 1:
                h[indf(i-2,j):indf(i-2,j+1), indf(i,j):indf(i,j+1)] = Lx_nnn_pos

            if i < Nx - 1:
                h[indf(i+1,j):indf(i+1,j+1), indf(i,j):indf(i,j+1)] = Lx_nn_neg

            if i < Nx - 2:
                h[indf(i+2,j):indf(i+2,j+1), indf(i,j):indf(i,j+1)] = Lx_nnn_neg

            if j > 0:
                h[indf(i,j-1):indf(i,j), indf(i,j):indf(i,j+1)] = Ly_nn_pos

            if j > 1:
                h[indf(i,j-2):indf(i,j-1), indf(i,j):indf(i,j+1)] = Ly_nnn_pos

            if j < Ny - 1:
                h[indf(i,j+1):indf(i,j+2), indf(i,j):indf(i,j+1)] = Ly_nn_neg

            if j < Ny - 2:
                h[indf(i,j+2):indf(i,j+3), indf(i,j):indf(i,j+1)] = Ly_nnn_neg

    flake_h = np.array(h).astype(np.complex128)

    return flake_h

In [ ]:
kx_sym, ky_sym = sp.symbols('k_x k_y', real = True)
alpha_sym = sp.symbols('alpha', real = True, positive = True)
beta_sym = sp.symbols('beta', real = True, positive = True)
gamma_z, lambda_z = sp.symbols('gamma_z lambda_z', real = True)

In [ ]:
s0 = sp.eye(2)
sx = sp.Matrix([[0, 1], [1, 0]])
sy = sp.Matrix([[0, -sp.I], [sp.I, 0]])
sz = sp.Matrix([[1, 0], [0, -1]])

In [ ]:
h_1d = (sp.sin(kx_sym) + alpha_sym * sp.sin(2*kx_sym)) * sx
h_1d += (sp.cos(kx_sym) + beta_sym * sp.cos(2*kx_sym)) * sz

In [ ]:
H_layered = sp.Matrix(np.zeros((4,4)))
H_layered += TensorProduct(sz,h_1d)
H_layered += TensorProduct(sy,s0) * lambda_z * sp.sin(ky_sym)
H_layered += TensorProduct(sx,s0) * (gamma_z + lambda_z * sp.cos(ky_sym))

In [ ]:
H_layered

In [ ]:
Nx = 29
Ny = 29
Nbands = 4
Nocc = 2

params = {}
params["Nx"] = Nx
params["Ny"] = Ny
params["Nbands"] = Nbands
params["Nocc"] = Nocc

In [ ]:
H_hodti_fixalpha = H_layered.subs({alpha_sym : 2.0, beta_sym: 2., gamma_z: 0.5, lambda_z: 1})

In [ ]:
flake_h = flake_geometry(H_hodti_fixalpha, (kx_sym, ky_sym), Nx, Ny, Nbands)

In [ ]:
vals, vecs = np.linalg.eigh(flake_h)

In [ ]:
ind = np.argsort(vals)
vals = vals[ind]
vecs = vecs[:,ind]

In [ ]:
plot_data = vals

fig = plt.figure(figsize = (4,3))
plt.scatter(np.arange(len(plot_data)), plot_data, c = "black", s = 10)
plt.xlabel("Eigenvalue index", fontsize = 16)
plt.ylabel(r"$E$", fontsize = 16, labelpad = -4)

plt.yticks([-3,0,3], ["-3", "0", "3"], fontsize = 14)
plt.xticks([0,1000,2000,3000], ["0", "1000", "2000", "3000"], fontsize = 14)
plt.show()

In [ ]:
mask = abs(vals) < 1e-1 * 5.2
plot_data = vals[mask]
plot_x = np.arange(len(vals))[mask]

fig = plt.figure(figsize = (1.5,1.5))
plt.scatter(plot_x, plot_data, c = "black", s = 10, alpha = 1)
plt.yticks(fontsize = 14)
plt.xticks(fontsize = 14)
plt.xticks([1682], ["1682"], fontsize = 14)
plt.show()

In [ ]:
density = np.zeros((Nx,Ny))

for state_index in range(Nx*Ny*2-4, Nx*Ny*2+4):
    for i in range(Nx):
        for j in range(Ny):
            density[i,j] += np.sum(np.abs(vecs[indf(i,j):indf(i,j+1),state_index])**2)

fig = plt.figure(figsize = (4,3))

plt.imshow(density.T, vmin = 0, vmax=1, origin = "lower")

plt.xticks(np.arange(Nx)[::7], fontsize = 14)
plt.yticks(np.arange(Ny)[::7], fontsize = 14)
plt.xlabel(r"$x$", fontsize = 16, labelpad = -1) 
plt.ylabel(r"$y$", fontsize = 16, labelpad = -0)
cbar = plt.colorbar()
cbar.set_ticks([0,0.5,1])    
cbar.set_ticklabels([r"$0$", r"$0.5$", r"$1$"], fontsize = 14)

plt.title(r"$\rho$", fontsize = 16, pad = 0.5)
plt.show()

In [ ]:
#the number of states in the plot is 
np.sum(density)